In [1]:
from selenium import  webdriver
import pandas as pd
import time
import datetime
link = "https://www.seek.co.nz/job/54719845?type=standout#searchRequestToken=e2219e7a-b251-43ba-9726-6df9f5fd0ef3"

In [2]:
# Read web driver
driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver.maximize_window()
driver.get(link)

In [3]:
# Get relevant contents from xpaths
job_title = driver.find_element_by_xpath("//*[@data-automation='job-detail-title']").text
company = driver.find_element_by_xpath("//*[@data-automation='advertiser-name']").text
section = driver.find_elements_by_xpath("//*[@class='yvsb870 _14uh9942u o76g430']")
location = section[0].text+"--"+section[1].text
field = section[2].text + "--" + section[3].text
posted = driver.find_element_by_xpath("//*[contains(text(),'Posted')]/span").text
_type = driver.find_element_by_xpath("//*[contains(text(),'ago')]").text#section[4].text
print("Title: {}\nCompany: {}\nLocation: {}\nField: {}\nType: {}\nPosted: {}".format(
    job_title, company, location, field, _type, posted
))

Title: Customer Insights / Data Scientist (Multiple roles)
Company: Customise Talent Group
Location: Auckland--Auckland Central
Field: Marketing & Communications--Market Research & Analysis
Type: 4d ago
Posted: 4d ago


In [7]:
# Get the whole text fromt he page to further analysis in future
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = link
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '--'.join(chunk for chunk in chunks if chunk)

In [4]:
# Read the running CSV file
df = pd.read_csv("seek_job_follow_up.csv")
df.tail()

,index,job_title,company,location,field,_type,posted,applied,text,link
29,seek_nz_0030,Dashboard Analyst,Banking Ombudsman Scheme,Wellington--Wellington Central,Information & Communication Technology--Busine...,17d ago,17d ago,2021-10-27,Dashboard Analyst Job in Wellington - SEEK--Da...,https://www.seek.co.nz/job/54293539?token=1~49...
30,seek_nz_0031,Data Analyst,Randstad Technologies,Wellington--Wellington Central,Information & Communication Technology--Other,27d ago,27d ago,2021-10-27,Data Analyst Job in Wellington - SEEK--Data An...,https://www.seek.co.nz/job/54154969?token=1~0d...
31,seek_nz_0032,Data Visualisation and Insight Analyst,Enable Networks Ltd,Canterbury--Christchurch,Information & Communication Technology--Busine...,21d ago,21d ago,2021-10-28,Data Visualisation and Insight Analyst Job in ...,https://www.seek.co.nz/job/54245209?token=1~49...
32,seek_nz_0033,Data Scientist,Narrative Muse,Auckland--Auckland Central,"Science & Technology--Mathematics, Statistics ...",3d ago,3d ago,2021-11-07,Data Scientist Job in Auckland - SEEK--Data Sc...,https://www.seek.co.nz/job/54697028?type=stand...
33,seek_nz_0034,Customer Insights / Data Scientist (Multiple r...,Customise Talent Group,Auckland--Auckland Central,Marketing & Communications--Market Research & ...,2d ago,2d ago,2021-11-07,Customer Insights / Data Scientist (Multiple r...,https://www.seek.co.nz/job/54719845?type=stand...


In [5]:
# Notify the last values form previous file
index = "seek_nz_"+str(df.shape[0]+1).zfill(4)
index

'seek_nz_0035'

In [9]:
# Create a dictionary with current details
temp = pd.DataFrame({
    "index": [index]
    ,"job_title": [job_title]
    ,"company": [company]
    ,"location": [location]
    ,"field": [field]
    ,"_type": [_type]
    ,"posted": [posted]
    ,"applied": [str(datetime.date.today())]
    ,"text": text
    ,"link": link
})
# Append above dictionary to the running file and remove duplicates
df = df.append(temp).drop_duplicates()
df.tail()

,index,job_title,company,location,field,_type,posted,applied,text,link
30,seek_nz_0031,Data Analyst,Randstad Technologies,Wellington--Wellington Central,Information & Communication Technology--Other,27d ago,27d ago,2021-10-27,Data Analyst Job in Wellington - SEEK--Data An...,https://www.seek.co.nz/job/54154969?token=1~0d...
31,seek_nz_0032,Data Visualisation and Insight Analyst,Enable Networks Ltd,Canterbury--Christchurch,Information & Communication Technology--Busine...,21d ago,21d ago,2021-10-28,Data Visualisation and Insight Analyst Job in ...,https://www.seek.co.nz/job/54245209?token=1~49...
32,seek_nz_0033,Data Scientist,Narrative Muse,Auckland--Auckland Central,"Science & Technology--Mathematics, Statistics ...",3d ago,3d ago,2021-11-07,Data Scientist Job in Auckland - SEEK--Data Sc...,https://www.seek.co.nz/job/54697028?type=stand...
33,seek_nz_0034,Customer Insights / Data Scientist (Multiple r...,Customise Talent Group,Auckland--Auckland Central,Marketing & Communications--Market Research & ...,2d ago,2d ago,2021-11-07,Customer Insights / Data Scientist (Multiple r...,https://www.seek.co.nz/job/54719845?type=stand...
0,seek_nz_0035,Customer Insights / Data Scientist (Multiple r...,Customise Talent Group,Auckland--Auckland Central,Marketing & Communications--Market Research & ...,4d ago,4d ago,2021-11-09,Customer Insights / Data Scientist (Multiple r...,https://www.seek.co.nz/job/54719845?type=stand...


In [119]:
# Save the file with new details
df.to_csv("seek_job_follow_up.csv", index=False)

In [120]:
# Close the web driver
driver.quit()